In [84]:
import torch
import random
import torch.nn as nn
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence, pack_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
import os

import pyarrow.parquet as pq

from tqdm import tqdm
from pprint import pprint
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from dateutil.parser import parse
import os

from sklearn.metrics import roc_auc_score

import keras

import tensorflow as tf

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, BatchSampler, Dataset

from IPython.display import display
import torch.nn.utils.rnn as rnn_utils
plt.style.use('fivethirtyeight')

In [81]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Model

In [82]:
class credGRU(nn.Module):
    def __init__(self, input_size, hidden_size, inter_size, output_size):
        super(credGRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.inter_size = inter_size
        self.output_size = output_size

        self.gru = nn.GRU(
            input_size,
            hidden_size,
            batch_first=True,
            bidirectional=False
        )
        self.linear1 = nn.Linear(self.hidden_size, self.inter_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(self.inter_size, self.output_size)

    def forward(self, input_list, lengths):
        out, _ = self.gru(input_list)
        out = self.linear1(out[torch.arange(out.size(0)), lengths])
        out = self.relu(out)
        out = self.linear2(out)

        return out


In [83]:
def OHEBool(x):
    return np.array([1 - x, x]).T.astype(float)

OHEBool(np.array([0,1]))
all_target = pd.read_csv('/content/drive/My Drive/Data/train_target.csv')

In [85]:
input_size = 59
hidden_size = 256
output_size = 2
batch_size = 16
inter_size = 20
num_epochs = 15

In [86]:
torch.cuda.is_available()

True

In [87]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [88]:
learning_rate = 0.001
model = credGRU(input_size, hidden_size, inter_size, output_size)
model = model.to(device)
loss_criterion  = nn.CrossEntropyLoss()
loss_criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [89]:
random_zeros_indices = np.random.choice(np.where(all_target['flag'] == 0)[0], size=500000, replace=False)
random_ones_indices = np.random.choice(np.where(all_target['flag'] == 1)[0], size=100000, replace=False)
selected_indices = np.concatenate((random_zeros_indices, random_ones_indices))
selected_indices.sort()
selected_indices

array([      3,       8,       9, ..., 2999984, 2999986, 2999991])

In [ ]:
for ii in range(num_epochs):

    for i in range(12):
        num_nil = i*250000
        sil = selected_indices[(selected_indices >= num_nil) & (selected_indices < num_nil + 250000)]
        tensors = pq.read_table('/content/drive/My Drive/Data/train_data/train_data_'+ str(i) + '.pq').to_pandas().drop('rn', axis=1)
        # ['flag'].to_numpy()

        target_nm = OHEBool(all_target['flag'].to_numpy())



        tensors = tensors.groupby(
            'id').apply(lambda x: x.drop('id', axis=1).values.tolist())


        tensors = [torch.tensor(lst).float() for lst in tensors.values]
        batch_num = 0
        current_loss = 0
        for j in range(0, len(tensors), batch_size):
            batch_num += 1
            input_list = tensors[j:j + batch_size]
            lengths = []
            for k in input_list:
                lengths.append(len(k) - 1)
            lengths = torch.tensor(lengths)
            input_list = rnn_utils.pad_sequence(input_list, batch_first=True)
            lengths = lengths.to(device)
            input_list = input_list.to(device)
            score = model(input_list, lengths)
            target = torch.from_numpy(target_nm[j:j+batch_size])
            target = target.to(device)
            loss = loss_criterion(score, target)
            current_loss += loss.item()

            # optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print(f"At epoch: {ii}/{num_epochs - 1}, file: {i},  loss: {current_loss/batch_num}")

At epoch: 0/14, file: 0,  loss: 0.18137151723239228
At epoch: 0/14, file: 1,  loss: 0.1492368452264387
At epoch: 0/14, file: 2,  loss: 0.14513703332229705
At epoch: 0/14, file: 3,  loss: 0.1472454562926758
At epoch: 0/14, file: 4,  loss: 0.14841460676220058
At epoch: 0/14, file: 5,  loss: 0.14738219499512947
At epoch: 0/14, file: 6,  loss: 0.1488502180168908
At epoch: 0/14, file: 7,  loss: 0.14515860815503448
At epoch: 0/14, file: 8,  loss: 0.14521307005118578
At epoch: 0/14, file: 9,  loss: 0.1473028067336902
At epoch: 0/14, file: 10,  loss: 0.14874143291198835
At epoch: 0/14, file: 11,  loss: 0.14610012469970807
At epoch: 1/14, file: 0,  loss: 0.14395990625567362
At epoch: 1/14, file: 1,  loss: 0.14350056680083276
At epoch: 1/14, file: 2,  loss: 0.14168360935138538
At epoch: 1/14, file: 3,  loss: 0.143422774039682
At epoch: 1/14, file: 4,  loss: 0.14127994314108788
At epoch: 1/14, file: 5,  loss: 0.14321751324956492
At epoch: 1/14, file: 6,  loss: 0.14559122898135335
At epoch: 1/14, 

In [ ]:
tensors = None
tensors1 = None
kek = []

In [ ]:
kk = 0
kek1 = None
model.eval()
tensors1 = pq.read_table('/content/drive/My Drive/Data/test_data/test_data_'+ str(kk) + '.pq').to_pandas().drop('rn', axis=1)

In [ ]:
for i in range(25):
    tensors = tensors1[tensors1['id'].isin(np.arange(3000000 + i*10000,3010000 + i*10000))]
    tensors = tensors.groupby(
            'id').apply(lambda x: x.drop('id', axis=1).values.tolist())
    tensors = [torch.tensor(lst).float() for lst in tensors.values]
    with torch.no_grad():
            lengths = []
            for k in tensors:
                lengths.append(len(k) - 1)
            lengths = torch.tensor(lengths)
            tensors = rnn_utils.pad_sequence(tensors, batch_first=True)
            kek1 = model(tensors.to(device), lengths.to(device))
            kek.append(kek1)

In [ ]:
tensors = None
tensors1 = None

In [ ]:
kk = 1
kek1 = None
tensors1 = pq.read_table('/content/drive/My Drive/Data/test_data/test_data_'+ str(kk) + '.pq').to_pandas().drop('rn', axis=1)

In [ ]:
for i in range(25):
    tensors = tensors1[tensors1['id'].isin(np.arange(3000000 +250000+ i*10000,3010000 + 250000+ i*10000))]
    tensors = tensors.groupby(
            'id').apply(lambda x: x.drop('id', axis=1).values.tolist())
    tensors = [torch.tensor(lst).float() for lst in tensors.values]
    with torch.no_grad():
            lengths = []
            for k in tensors:
                lengths.append(len(k) - 1)
            lengths = torch.tensor(lengths)
            tensors = rnn_utils.pad_sequence(tensors, batch_first=True)
            kek1 = model(tensors.to(device), lengths.to(device))
            kek.append(kek1)

In [ ]:
out_list = []
for i in kek:
    out_list.append(nn.Softmax()(i).T[1])
concatenated_tensor = torch.cat(out_list)
ids = np.arange(3000000, 3500000)

# Создаем DataFrame
ansdf = pd.DataFrame({'id': ids, 'score': concatenated_tensor.to('cpu')})
ansdf.to_csv('/content/drive/My Drive/Data/res.csv', index=False)